# Env

In [ ]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234,
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

# Entropy

In [ ]:
# Generate linearly spaced vector
p = np.linspace(0, 1, 101)[1:-1]
q = 1 - p
prob = np.stack([p, q], axis=-1)

print(prob)

In [ ]:
# entropy
entropy = - prob * np.log2(prob)
entropy = np.sum(entropy, axis=-1)
entropy

In [ ]:
# draw plot
plt.plot(p, entropy)
plt.title('Entropy')
plt.show()

# Cross Entropy

In [ ]:
# Generate linearly spaced vector
p = np.linspace(0, 1, 11)[1:-1]
q = p
print(p, q)

In [ ]:
cross_entropy = np.zeros((9, 9))
for i, p_a in enumerate(p):
    p_b = 1 - p_a
    for j, q_a in enumerate(q):
        q_b = 1 - q_a
        ce = -(p_a * np.log(q_a) + p_b * np.log(q_b))
        cross_entropy[i][j] = np.round(ce, 2)

print(cross_entropy)

# Corss Entropy Loss

In [ ]:
y_true = [[1, 0, 0, 0]] * 4 + [[0, 1, 0, 0]] * 3 + [[0, 0, 1, 0]] * 2 + [[0, 0, 0, 1]] * 1
y_true = np.array(y_true)
y_true

In [ ]:
y_pred = [[0.25, 0.25, 0.25, 0.25]] * 10
y_pred = np.array(y_pred)
y_pred

$-\log \hat{y}_{ij}$ 값

In [ ]:
y_log_pred = -np.log(y_pred)
y_log_pred

$-y_{ij} \log \hat{y}_{ij}$ 값

In [ ]:
ce_item = y_true * y_log_pred
ce_item

$-\sum_{j=1}^C y_{ij} \ \log \hat{y}_{ij}$ 값

In [ ]:
ces = np.sum(ce_item, axis=-1)
ces

$-{1 \over N} \sum_{i=n}^N\sum_{j=1}^C y_{ij} \ \log \hat{y}_{ij}$

In [ ]:
loss = np.mean(ces)
loss

In [ ]:
y_pred = [[0.40, 0.30, 0.20, 0.10]] * 10
y_pred = np.array(y_pred)
y_pred

$-\log \hat{y}_{ij}$ 값

In [ ]:
y_log_pred = -np.log(y_pred)
y_log_pred

$-y_{ij} \log \hat{y}_{ij}$ 값

In [ ]:
ce_item = y_true * y_log_pred
ce_item

$-\sum_{j=1}^C y_{ij} \ \log \hat{y}_{ij}$ 값

In [ ]:
ces = np.sum(ce_item, axis=-1)
ces

$-{1 \over N} \sum_{i=n}^N\sum_{j=1}^C y_{ij} \ \log \hat{y}_{ij}$

In [ ]:
loss = np.mean(ces)
loss

# 실습
- 10개 짜리 두개의 확률분포 y_true, y_pred를 선언하고 두 확률분포의 cross-entropy loss를 직접 계산하세요.

# TF Cross Entropy

In [ ]:
y_true = np.random.randint(0, 4, (10,))
y_true

In [ ]:
y_prob = np.zeros((10, 4))
y_prob

In [ ]:
for i, y in enumerate(y_true):
    y_prob[i, y] = 1
y_prob

In [ ]:
logits = np.random.randn(10, 4)
logits

In [ ]:
y_pred = tf.nn.softmax(logits)
y_pred

In [ ]:
ce1 = -y_prob * np.log(y_pred)
ce1

In [ ]:
ce2 = np.sum(ce1, axis=-1)
ce2

In [ ]:
ce = np.mean(ce2)
ce

In [ ]:
tf.keras.losses.CategoricalCrossentropy()(y_prob, y_pred)

In [ ]:
tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_prob, logits)

In [ ]:
tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)

In [ ]:
tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(y_true, logits)

# 실습
- 긍정, 부정 두가지를 예측하는 10개의 sample의 정답 y_true와 logits을 선언하고 아래 형식에 따른 cross entropy loss를 구하세요.
  - 직접계산
  - tf.keras.losses.CategoricalCrossentropy()
  - tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  - tf.keras.losses.SparseCategoricalCrossentropy()
  - tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)